In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("luohy/SAIL-7b")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "luohy/SAIL-7b",
    device_map="auto",  # Automatically distribute across available GPUs
    torch_dtype=torch.float16  # Use half precision to reduce memory usage
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(model.hf_device_map)

{'': 0}


now testing on hate speech

In [5]:
!pip install --upgrade transformers datasets
!pip install openai==0.28

In [ ]:
import openai
import torch
import re
import os
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, f1_score

# 1. Load the dataset
!git clone https://github.com/Vicomtech/hate-speech-dataset.git
dataset_path = "hate-speech-dataset/sampled_test"
annotations_path = "hate-speech-dataset/annotations_metadata.csv"

annotations_df = pd.read_csv(annotations_path)
annotations_df = annotations_df[["file_id", "label"]]

label_map = {"noHate": 0, "hate": 1}
annotations_df["label"] = annotations_df["label"].map(label_map)
label_dict = dict(zip(annotations_df["file_id"], annotations_df["label"]))

def load_text_files(folder_path, label_dict):
    dataset = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_id = filename.replace(".txt", "")

            if file_id in label_dict:
                file_path = os.path.join(folder_path, filename)
                with open(file_path, "r", encoding="utf-8") as file:
                    text = file.read().strip()
                    dataset.append({"text": text, "label": label_dict[file_id]})
    return dataset

processed_ds = load_text_files(dataset_path, label_dict)

print(f"Loaded {len(processed_ds)} samples from {dataset_path}")
print("Sample data:", processed_ds[:5])








print("Loading SAIL-7B model...")
openai.api_key = ""
openai.api_base = "https://cmu.litellm.ai/v1"

def create_fairness_prompt(claim):
    return f"Is it fair to say that? {claim}"

def extract_yes_no(response_text):
    match = re.search(r'(?i)(yes|no)', response_text)
    if match:
        return 0 if match.group(1).lower() == "yes" else 1  # yes: fair (0), no: unfair (1)
    return 1

def think_twice(claim, model, tokenizer):
    prompt_suspicious = f"Find the most suspicious part of the statement:\n\n{claim}\n\nSuspicious Part:"
    # 1.
    inputs = tokenizer(prompt_suspicious, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=20)
    suspicious_info = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("\n🔹 Suspicious Info:", suspicious_info)

    # 2.
    prompt_fact = f"Given the statement:\n\n{claim}\n\nThe most suspicious part is: {suspicious_info}\n\nProvide a relevant social or scientific fact to verify it:\nFact:"
    inputs = tokenizer(prompt_fact, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=30)
    fact = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("🔹 Generated Fact:", fact)

    # 3.
    final_prompt = f"Claim: {claim}\nThe claim mentions that {suspicious_info}.\nFact: {fact}\nQuestion: Is it fair to say that?\nAnswer:"
    inputs = tokenizer(final_prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=5)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("🔹 Final Answer:", response)
    return extract_yes_no(response)  # yes -> 0 (fair), no -> 1 (unfair)

def classify_fairness_sail(claim, model, tokenizer):
    return think_twice(claim, model, tokenizer)

# def classify_fairness_sail(claim, model, tokenizer):
#     prompt = create_fairness_prompt(claim)
#     inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
#     with torch.no_grad():
#         outputs = model.generate(**inputs, max_new_tokens=5)
#     response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     print("SAIL-7B output:", response)
#     return extract_yes_no(response)

def classify_fairness_gpt(claim):
    prompt = create_fairness_prompt(claim)
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are a helpful assistant."},
                  {"role": "user", "content": prompt}],
        max_tokens=50,
        temperature=0.7
    )
    gpt_response_text = response["choices"][0]["message"]["content"]
    print("GPT-4o output:", gpt_response_text)
    return extract_yes_no(gpt_response_text)

torch.manual_seed(42)
predictions_sail = []
ground_truths = []


# num_samples = 100
# sampled_ds = ds.shuffle(seed=42).select(range(num_samples))

print("Running inference...")
# for example in sampled_ds:
for example in processed_ds:
    claim_text = example["text"]
    # ground_truth_label = example["label"]
    # ground_truths.append(ground_truth_label)

    gt = classify_fairness_gpt(claim_text)
    ground_truths.append(gt)

    pred = classify_fairness_sail(claim_text, model, tokenizer)
    predictions_sail.append(pred)

    print(f"\nClaim: {claim_text}")
    print(f"Ground Truth (Dataset Label): {'FAIR' if gt == 0 else 'UNFAIR'}")
    print(f"SAIL-7B Prediction: {'FAIR' if pred == 0 else 'UNFAIR'}")
    print("-" * 50)


acc = accuracy_score(ground_truths, predictions_sail)
f1 = f1_score(ground_truths, predictions_sail, average='macro')

print("\n🔹 SAIL-7B Accuracy vs GPT-4o-mini Ground Truth:", acc)
print("🔹 SAIL-7B F1 Score vs GPT-4o-mini Ground Truth:", f1)


Streaming output truncated to the last 5000 lines.
The problems start when you start to accommonadate these people in the first place ! , give an inch and they 'll take a mile ......

Suspicious Part: "The problems start when you start to accommonadate these people in the first place ! "

Provide a relevant social or scientific fact to verify it:
Fact: Accommodation refers to the act of providing a place for someone to stay, such as a hotel or a room, or to provide for the
🔹 Final Answer: Claim: The problems start when you start to accommonadate these people in the first place ! , give an inch and they 'll take a mile ......
The claim mentions that Find the most suspicious part of the statement:

The problems start when you start to accommonadate these people in the first place ! , give an inch and they 'll take a mile ......

Suspicious Part: "The problems start when you start to accommonadate these people in the first place ! ".
Fact: Given the statement:

The problems start when you